In [15]:
import os
os.getcwd()
os.chdir("C:/Users/Devyani/Desktop/ML ASSIGNMENT")
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [16]:
df=pd.read_csv("Admission_Predict_Ver1.1.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [17]:
df=df.loc[:, ~df.columns.str.contains('^Serial No.')]
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [18]:
df.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [19]:
df.dtypes

GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [20]:
df.columns = ['GRE_Score','TOEFL_Score','University_Rating','SOP','LOR','CGPA','Research','Chance_of_Admit']
df.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [27]:
print(df.Research.value_counts())

1    280
0    220
Name: Research, dtype: int64


In [28]:
X= df.drop(["Chance_of_Admit"],1)
y=df["Chance_of_Admit"]

In [30]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols = list(X)
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=cols)

In [31]:
X.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research
0,0.94,0.928571,0.75,0.875,0.875,0.913462,1.0
1,0.68,0.535714,0.75,0.750,0.875,0.663462,1.0
2,0.52,0.428571,0.50,0.500,0.625,0.384615,1.0
3,0.64,0.642857,0.50,0.625,0.375,0.599359,1.0
4,0.48,0.392857,0.25,0.250,0.500,0.451923,0.0


In [33]:
import statsmodels.api as sm
cols = list(X)
p_max = 1
while (len(cols)>0):
    p=[]
    X_1= X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = list(model.pvalues.values[1:])
    p = pd.Series(p,index = cols)
    p_max = max(p)
    features_with_p_max = p.idxmax()
    if (p_max>0.05):
        cols.remove(features_with_p_max)
    else:
        break
selected_features = cols

In [34]:
selected_features
#Feature Selected columns are not used as only 2 columns get eliminated out of 7, Therefore all columns are used

['GRE_Score', 'TOEFL_Score', 'LOR', 'CGPA', 'Research']

In [35]:
#Splitting the data in train and test
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2,random_state=42)

In [40]:
from sklearn.metrics import r2_score, mean_squared_error

In [37]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg= LinearRegression()

lr_model= lin_reg.fit(X_train,y_train)
y_pred_lr= lr_model.predict(X_test)
r2_score(y_test,y_pred_lr)

0.818843256782963

In [38]:
#Decision Tree
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state = 42)

dt_model= dt.fit(X_train,y_train)
y_pred_dt= dt_model.predict(X_test)
r2_score(y_test,y_pred_dt)

0.5767726161369194

In [39]:
#Random Forrest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

rf_model= rf.fit(X_train,y_train)
y_pred_rf= rf_model.predict(X_test)
r2_score(y_test,y_pred_rf)

0.7918829437652809

In [42]:
#KNN
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()

knn.fit(X_train,y_train)
y_pred_knn=knn.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred_knn))
r2_score(y_test,y_pred_knn)

0.7589476772616137